In [1]:
import pandas as pd
import numpy as np

In [3]:
# Define data types for initial parsing of CSV
dtype = {
    'id': np.int64,
    'listing_id': np.int64,
    'date': str,
    'reviewer_id': np.int64,
    'comments': str
}
     
# Specify columns that should be parsed as dates
parse_dates = ['date']

# Specify conversion functions for particular columns
converters = {
}

In [4]:
# Read CSV contents into dataframe
df = pd.read_csv(
    '../res/reviews-3-aug-16.csv.gz', 
    usecols=list(dtype.keys()),
    dtype=dtype, 
    parse_dates=parse_dates, 
    converters=converters
)

In [5]:
# Check for columns containing null values
df.isnull().any()

listing_id     False
id             False
date           False
reviewer_id    False
comments        True
dtype: bool

In [5]:
# # Fill certain null / missing values
# df['reviews_per_month'].fillna(value=0.0, inplace=True)
# df['bathrooms'].fillna(value=0.0, inplace=True)
# df['bedrooms'].fillna(value=0.0, inplace=True)

# # Drop entries with certain missing values
# df.dropna(subset=['id', 'description', 'host_since', 'host_name'], inplace=True)

In [8]:
# Delete all existing Reviews
Review.objects.all().delete()

# Instantiate and save a Listing for each row in the dataframe
for index, row in df.iterrows():
    try:
        listing = Listing.objects.get(pk=row['listing_id'])
        Review(
            listing=listing,
            id=row['id'],
            date=row['date'],
            reviewer_id=row['reviewer_id'],
            comments=row['comments']
        ).save() # Commit to DB
    except Listing.DoesNotExist:
        print ("Listing %d does not exist" % row['listing_id'])
    if index % 1000 == 0:
        print('.', end='')

..........................................................................................................................................................................................................................................................Listing 2898119 does not exist
Listing 2898119 does not exist
.................Listing 10034835 does not exist
Listing 10034835 does not exist
...........................................................................................................................................Listing 6505664 does not exist
........................Listing 9998970 does not exist
..........

In [11]:
Review.objects.filter(comments="nan").update(comments="")

206

In [15]:
from django.db.models.functions import ExtractMonth
Review.objects.annotate(month=ExtractMonth('date')).filter(month=6).count()

53344

In [52]:
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct','Nov','Dec']
def print_monthly_review_totals(queryset):
    for i in range(1,13):
        print(months[i-1], '*', queryset.annotate(month=ExtractMonth('date')).filter(month=i).count() / queryset.count())

In [53]:
print_monthly_review_totals(Review.objects.all())

Jan * 0.06774114262826349
Feb * 0.07311883261114954
Mar * 0.08771119324181627
Apr * 0.09739467647380111
May * 0.1116047045115246
Jun * 0.12139970141645122
Jul * 0.13941484906965737
Aug * 0.07480519244073845
Sep * 0.05942540873174817
Oct * 0.06226104212941048
Nov * 0.052522939955576595
Dec * 0.052600316789862725


In [47]:
listing_ids = [_['id'] for _ in Neighborhood.objects.get(name="Van Nuys").listing_set.all().values('id')]
print_monthly_review_totals(Review.objects.filter(listing_id__in=listing_ids))

Jan * 0.059813084112149535
Feb * 0.052336448598130844
Mar * 0.07850467289719626
Apr * 0.10654205607476636
May * 0.10841121495327102
Jun * 0.1485981308411215
Jul * 0.18037383177570093
Aug * 0.056074766355140186
Sep * 0.058878504672897194
Oct * 0.04579439252336449
Nov * 0.05327102803738318
Dec * 0.0514018691588785


In [3]:
for n in Neighborhood.objects.all():
    print(n.name, n.computed_stats['avg_estimated_bookings_per_listing'])

Hasley Canyon None
Angeles Crest 30.0
Arcadia 8.928571428571429
Beverly Hills 23.119760479041915
Castaic 6.0
Castaic Canyons 24.615384615384617
Azusa 7.5
Cerritos 19.333333333333332
Chatsworth 9.666666666666666
Calabasas 15.0
Carson 49.54838709677419
Culver City 41.943127962085306
Claremont 39.611111111111114
Compton 1.3076923076923077
Covina 15.384615384615385
East San Gabriel 15.612903225806452
Florence 2.5
East Compton 0.0
East Los Angeles 40.18181818181818
El Monte 22.88888888888889
Elysian Valley 21.7
Glendale 24.787644787644787
Harbor Gateway 22.0
Hancock Park 38.15238095238095
La Cañada Flintridge 28.125
Montecito Heights 25.1
Irwindale 10.666666666666666
Jefferson Park 25.565217391304348
La Verne 16.2
La Mirada 30.0
Lancaster 14.923076923076923
Long Beach 35.46650124069479
Northeast Antelope Valley 0.0
Malibu 29.70769230769231
Palmdale 8.571428571428571
Monrovia 18.533333333333335
Montebello 22.42105263157895
Northwest Palmdale 2.0
Palos Verdes Estates 13.454545454545455
Pasade